In [5]:
import sqlite3
import pandas as pd
import datetime

conn = sqlite3.connect('../database.sqlite')
query_matches = \
    'select match_api_id, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal, date \
    from Match;'
matches = pd.read_sql(query_matches, conn, index_col='match_api_id', parse_dates=['date']);
matches.head()

,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,date
match_api_id,,,,,
492473,9987,9993,1,1,2008-08-17
492474,10000,9994,0,0,2008-08-16
492475,9984,8635,0,3,2008-08-16
492476,9991,9998,5,0,2008-08-17
492477,7947,9985,1,3,2008-08-16


In [6]:
import pandas as pd
def teamattrs(team_api_id, matchdate):
    """
    Helper method that gets the specified team's attributes from
    the date closest to matchdate.
    
    Keyword arguments:
    team_api_id -- the api_id of the team of interest (Integer)
    matchdate -- pd.Timestamp
    """
    query_attrs = \
        'select date, buildUpPlaySpeed, buildUpPlayPassing, \
                chanceCreationPassing, chanceCreationCrossing, chanceCreationShooting, \
                defencePressure, defenceAggression, defenceTeamWidth \
        from Team_Attributes \
        where team_api_id = %s \
        order by date asc;' % team_api_id
    attrs = pd.read_sql(query_attrs, conn, parse_dates=['date'])
    if (not attrs.empty):
        closestattrs = None
        mindiff = None
        for index, row in attrs.iterrows():
            date = row['date']
            diff = abs(matchdate - date)
            if (mindiff is None or diff < mindiff):
                mindiff = diff
                closestattrs = row
        return closestattrs.drop('date')
    else:
        return None
    
    
def get_winner(match):
    """
    Given a pd.Series representing a match, returns the 
    team_api_id of the winning team.
    If the match was a draw, returns None.
    """
    if match['home_team_goal'] > match['away_team_goal']:
        return match['home_team_api_id']
    elif match['home_team_goal'] < match['away_team_goal']:
        return match['away_team_api_id']
    else:
        return None

def get_win_percentage(encounters, team_api_id):
    """
    Given a pd.DataFrame of encounters between two teams,
    extracts the win percentage of the specified team.
    
    For each match, if it was a draw, counts it as a
    win for *both* teams (and so increases the total count
    by 2).
    
    If there were no encounters between the two teams,
    returns -1.
    """
    total_count = len(encounters.index)
    if total_count == 0:
        return -1
    
    wins = 0
    for (name, item) in encounters.iterrows():
        winner = get_winner(item)
        if winner is None:
            wins = wins + 1
            total_count = total_count + 2
        elif winner == team_api_id:
            wins = wins + 1
        
    return wins / total_count

In [7]:
def differencevector(row):
    """
    This function is applied to each row in the 'matches' DataFrame.
    For each row, it returns a pandas.Series containing the number of
    goals scored by each team and the result from the home team's
    perspective.
    
    Keyword arguments:
    row -- pandas.Series
    """
    date = row['date']
    homegoal = row['home_team_goal']
    homeattrs = teamattrs(row['home_team_api_id'], date)
    
    awaygoal = row['away_team_goal']
    awayattrs = teamattrs(row['away_team_api_id'], date)
    
    match_query = \
        "select date, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal \
        from `Match` \
        where date < '%s' and (home_team_api_id in ('%s', '%s') and (away_team_api_id in ('%s', '%s')))" % \
        (str(row['date']) + ' 00:00:00', row['home_team_api_id'], row['away_team_api_id'], row['home_team_api_id'], row['away_team_api_id'])
    encounters = pd.read_sql(match_query, conn, parse_dates=['date'])
    
    win_percentage = get_win_percentage(encounters, row['home_team_api_id'])
    
    if (homeattrs is not None and awayattrs is not None):
        differencevector = homeattrs - awayattrs

        if (homegoal > awaygoal):
            result = 2
        elif (awaygoal == homegoal):
            result = 1
        else:
            result = 0

        results = pd.Series({
            'outcome':  result,
            'win_percentage': win_percentage
        })
        return results.append([differencevector])
    else:
        return

results = matches.apply(differencevector, axis=1).dropna(axis=0, how='any')
conn.close()

results.to_csv('../difference_vectors.csv')